# GBA 6430 Assignment 4: Spark
# Dr. Mohammad Salehan

- In this assignment, you will conduct Spark transformations on different RDDs. 
- You should complete this assignment using RDD. 
- Use of DataFrame is not allowed.

1. Store SalesJan2009.csv in S3 and load it into a RDD. Then retrieve 5 elements from the RDD.

In [22]:
myRDD =(
    sc
    .textFile('s3://gba6430-huayang-01/intro/SalesJan2009.csv', minPartitions=8)
    .map(lambda line: line.split(","))
)
myRDD.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['Transaction_date', 'Product', 'Price', 'Payment_Type', 'Name', 'City', 'State', 'Country', 'Account_Created', 'Last_Login', 'Latitude', 'Longitude'], ['1/2/2009 6:17', 'Product1', '1200', 'Mastercard', 'carolina', 'Basildon', 'England', 'United Kingdom', '1/2/2009 6:00', '1/2/2009 6:08', '51.5', '-1.1166667'], ['1/2/2009 4:53', 'Product1', '1200', 'Visa', 'Betina', 'Parkville                   ', 'MO', 'United States', '1/2/2009 4:42', '1/2/2009 7:49', '39.195', '-94.68194'], ['1/2/2009 13:08', 'Product1', '1200', 'Mastercard', 'Federica e Andrea', 'Astoria                     ', 'OR', 'United States', '1/1/2009 16:21', '1/3/2009 12:32', '46.18806', '-123.83'], ['1/3/2009 14:44', 'Product1', '1200', 'Visa', 'Gouya', 'Echuca', 'Victoria', 'Australia', '9/25/2005 21:13', '1/3/2009 14:22', '-36.1333333', '144.75']]

2. Removes the header of the CSV file from the RDD. Use the resulting RDD for the future steps.

In [23]:
header = myRDD.first()
myRDD = myRDD.filter(lambda row : row != header)
myRDD.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['1/2/2009 6:17', 'Product1', '1200', 'Mastercard', 'carolina', 'Basildon', 'England', 'United Kingdom', '1/2/2009 6:00', '1/2/2009 6:08', '51.5', '-1.1166667'], ['1/2/2009 4:53', 'Product1', '1200', 'Visa', 'Betina', 'Parkville                   ', 'MO', 'United States', '1/2/2009 4:42', '1/2/2009 7:49', '39.195', '-94.68194'], ['1/2/2009 13:08', 'Product1', '1200', 'Mastercard', 'Federica e Andrea', 'Astoria                     ', 'OR', 'United States', '1/1/2009 16:21', '1/3/2009 12:32', '46.18806', '-123.83'], ['1/3/2009 14:44', 'Product1', '1200', 'Visa', 'Gouya', 'Echuca', 'Victoria', 'Australia', '9/25/2005 21:13', '1/3/2009 14:22', '-36.1333333', '144.75'], ['1/4/2009 12:56', 'Product2', '3600', 'Visa', 'Gerd W ', 'Cahaba Heights              ', 'AL', 'United States', '11/15/2008 15:47', '1/4/2009 12:45', '33.52056', '-86.8025']]

3. Displays number of records.

In [24]:
myRDD.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

998

4. Select the transactions paid by Visa. Display 5 records.

In [25]:
myRDD\
.filter(lambda c: c[3] == "Visa")\
.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['1/2/2009 4:53', 'Product1', '1200', 'Visa', 'Betina', 'Parkville                   ', 'MO', 'United States', '1/2/2009 4:42', '1/2/2009 7:49', '39.195', '-94.68194'], ['1/3/2009 14:44', 'Product1', '1200', 'Visa', 'Gouya', 'Echuca', 'Victoria', 'Australia', '9/25/2005 21:13', '1/3/2009 14:22', '-36.1333333', '144.75'], ['1/4/2009 12:56', 'Product2', '3600', 'Visa', 'Gerd W ', 'Cahaba Heights              ', 'AL', 'United States', '11/15/2008 15:47', '1/4/2009 12:45', '33.52056', '-86.8025'], ['1/4/2009 13:19', 'Product1', '1200', 'Visa', 'LAURENCE', 'Mickleton                   ', 'NJ', 'United States', '9/24/2008 15:19', '1/4/2009 13:04', '39.79', '-75.23806'], ['1/4/2009 14:11', 'Product1', '1200', 'Visa', 'Aidan', 'Chatou', 'Ile-de-France', 'France', '6/3/2008 4:22', '1/5/2009 1:17', '48.8833333', '2.15']]

5. Calculate total sales for each payment method. Note that all values in the RDD are string and you need to convert price to integer.

In [26]:
myRDD.map(lambda c: (c[3], int(c[2])))\
.reduceByKey(lambda x, y: x + y)\
.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('Diners', 133800), ('Mastercard', 458450), ('Amex', 188900), ('Visa', 849350)]

6. Calculate cashback for each payment method. Here is the cashback percentage for each method: <br>
Visa	1%<br>
Mastercard	2%<br>
Diners	3%<br>
Amex	4%<br>
Sort the results using cashback amount in descending order.

In [27]:
cashbackPer={
    "Visa": 0.01,
    "Mastercard": 0.02,
    "Diners": 0.03,
    "Amex": 0.04
}
a =myRDD.map(lambda c: (c[3], int(c[2])))\
.reduceByKey(lambda x, y: x + y)
a.map(lambda x: (x[0], x[1] * cashbackPer.get(x[0], 0.0)))\
.sortBy(lambda x: x[1], ascending=False)\
.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('Mastercard', 9169.0), ('Visa', 8493.5), ('Amex', 7556.0), ('Diners', 4014.0)]

7. Calculate the distinct values for Product. Make sure spaces in the beginning or end of product names do not create new products.

In [28]:
myRDD.map(lambda fields: fields[1].strip())\
.distinct()\
.collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Product3', 'Product2', 'Product1']